In [2]:
import cv2
import numpy as np
from glob import glob
np.set_printoptions(suppress=True, formatter={'float_kind':'{:f}'.format})

In [3]:
def get_line(coords):
    assert np.shape(coords)[0] == 2
    x1 = np.hstack((coords[0],1))
    x2 = np.hstack((coords[1],1))
    l = np.cross(x1,x2)
    l = np.divide(l,l[-1])
    return l
   
def find_line_infinity(coords):
    assert np.shape(coords)[0] == 4
    l1,l2 = get_line(coords[:2]), get_line(coords[2:])
    p1 = np.cross(l1,l2)
    p1 = np.divide(p1,p1[-1])
    l3,l4 = get_line(coords[::2]), get_line(coords[1::2])
    p2 = np.cross(l3,l4)
    p2 = np.divide(p2,p2[-1])
    l_inf = np.cross(p1,p2)
    l_inf = np.divide(l_inf,l_inf[-1])
   
    return  p1,p2,l_inf

In [4]:
def build_k_matrix(kkt):
    k = np.zeros((3,3))
    k[:,-1] = kkt[:,-1]
    k[1,1] = np.sqrt(kkt[1,1]-kkt[1,2]**2)
    k[0,1] = (kkt[0,1] - kkt[0,2]*kkt[1,2])/k[1,1]
    k[0,0] = np.sqrt(kkt[0,0]-k[0,1]**2-kkt[0,2]**2)
    k[-1,-1] = 1
    return k

In [47]:
#Dataset1  
#dimension of images 750X900
COORDS_1 = [[(55,111),(401,152),(62,520),(456,485)],
              [(124,237),(363,106),(151,531),(430,486)],
              [(156,266),(355,131),(175,546),(406,522)],
              [(96,219),(395,85),(142,541),(490,467)],
              [(161,294),(428,149),(266,600),(441,419)],
              [(176,13),(473,183),(137,510),(499,530)],
              ]

#Dataset2 
#Dimension of image 768*1024
COORDS_2 = [[(247,543),(512,552),(171,710),(579,725)],\
          [(318,575),(589,604),(177,705),(590,766)],\
          [(164,215),(449,335),(176,676),(485,659)],\
          [(294,391),(565,310),(223,680),(549,672)],\
          [(211,323),(541,327),(157,639),(603,639)],\
          [(385,516),(653,565),(210,623),(580,724)]]

x=len(COORDS_1)
A = np.zeros((x,4))
i = 0
    
for k in range (x):
    coords = COORDS_1[k]
    p1,p2,l_inf = find_line_infinity(coords)
    A[i] = np.asarray([p1[0]*p2[0]+p1[1]*p2[1],1,p1[0]+p2[0],p1[1]+p2[1]])
    i += 1

_,_,vh = np.linalg.svd(A)
x = vh.T[:,-1]

w = np.asarray([[x[0],0,x[2]],[0,x[0],x[3]],[x[2],x[3],x[1]]])
kkt = np.linalg.pinv(w)
kkt = np.divide(kkt,kkt[-1,-1])
k = build_k_matrix(kkt)

print("For Dataset2")
print('Intrinsic Parameter of Camera k:')
print(k)
print('\n Image of Absolute Conic w: \n')
w = np.linalg.inv(np.matmul(k,k.T))
print(np.divide(w,w[-1,-1]))


For Dataset2
Intrinsic Parameter of Camera k:
[[596.814103 0.000000 325.509371]
 [0.000000 596.814103 414.169660]
 [0.000000 0.000000 1.000000]]

 Image of Absolute Conic w: 

[[0.000002 -0.000000 -0.000514]
 [-0.000000 0.000002 -0.000654]
 [-0.000514 -0.000654 1.000000]]


In [70]:
#inverse of K 
kin=np.linalg.pinv(k)
kin = np.divide(kin,kin[-1,-1])
kin

array([[0.001676, -0.000000, -0.545412],
       [-0.000000, 0.001676, -0.693968],
       [0.000000, -0.000000, 1.000000]])

# Finding Extrinsic parameter of camera

In [71]:
img_points2 = np.array([[205,255,1],[229,256,1],
              [278,255,1],[207,307,1],
              [230,307,1],[277,305,1]])

w_points2= np.array([[0.6,12.1,64.9],[3.2,12.1,64.9],
              [8.2,12.1,64.9],[0.6,6.9,64.9],
              [3.2,6.9,64.9],[8.2,6.9,64.9]])

# img_points2 = np.array([[108,476,1],[176,479,1],[309,477,1],[106,615,1],
#                 [174,615,1],[310,615,1]])

# w_points2= np.array([[0.6,12.1,26.3],[3.2,12.1,26.3],
#               [8.2,12.1,26.3],[0.6,6.9,26.3],
#               [3.2,6.9,26.3],[8.2,6.9,26.3]])


In [72]:
a=3*len(img_points2)
A= np.zeros((a,12))
b= np.zeros((a,1))
    
for i in range(0,a,3):
    #print(i)
    X = w_points2[int(i/3)][0]
    Y=  w_points2[int(i/3)][1]
    Z=  w_points2[int(i/3)][2]
    #print(i)
    #Forming set of linear homogeneous equation(by Direct linear Transform)
    A[i]=[X,Y,Z,0,0,0,0,0,0,1,0,0]
    A[i+1]=[0,0,0,X,Y,Z,0,0,0,0,1,0]
    A[i+2]=[0,0,0,0,0,0,X,Y,Z,0,0,1]
    
    
    
    p=[img_points2[int(i/3)][0],img_points2[int(i/3)][1],img_points2[int(i/3)][2]]
    #print(p)
    
    c = np.matmul(kin,np.transpose(p))
    b[i]=c[0]
    b[i+1]=c[1]
    b[i+2]=c[2]
    
    #i=i+3
   
rt= np.linalg.lstsq(A,b,rcond=-1)[0]
 
#Extrinsic parameter matrix rot_trans = [R t]
shape=(3,4)
rot_trans=np.zeros(shape)
j=0
for i in range(3):
    
    rot_trans[i][0] = rt[j+0]
    rot_trans[i][1] = rt[j+1]
    rot_trans[i][2] = rt[j+2]
    #rot_trans[i][3]  = rt[j+3]
    j=j+3
rot_trans[0][3]=rt[9]
rot_trans[1][3]=rt[10]
rot_trans[2][3]=rt[11]

print("Extrinsic Parameter Matrix for DataSet2")
print(rot_trans)     

Extrinsic Parameter Matrix for DataSet2
[[0.015807 -0.000215 -0.003210 -0.000049]
 [-0.000258 -0.016433 -0.001021 -0.000016]
 [0.000000 -0.000000 0.015405 0.000237]]


In [73]:
projection_matrix=np.matmul(k,rot_trans)
P=projection_matrix
print("Projection Matrix For Dataset 2 :\n")
print(P)

Projection Matrix For Dataset 2 :

[[9.433646 -0.128205 3.098648 0.047745]
 [-0.154155 -9.807692 5.770942 0.088921]
 [0.000000 -0.000000 0.015405 0.000237]]


# Projecting an Artificial Object into image plane from world coordinate

In [88]:
img1 = cv2.imread('./NewData1/chess_vr.jpg')
img1 = cv2.resize(img1,(600,900))
color = (0, 125, 255)
thickness =2
# world coordinate point of a Prism 
#dataset2
# a = np.array([5.7,12.3,26.3,1])
# b = np.array([10.8,12.3,26.3,1])
# c = np.array([10.8,6.8,26.3,1])
# d = np.array([5.7,6.8,26.3,1])
# e = np.array([15.2,9.5,26.3,1])

#dataset1

a = np.array([5.7,12.3,64,1])
b = np.array([10.8,12.3,64,1])
c = np.array([10.8,6.8,64,1])
d = np.array([5.7,6.8,64,1])
e = np.array([15.2,9.5,64,1])


#projected world coordinate of cube into image plane by doing
# x=PX operation
projected_a =np.matmul(P,a)
projected_b=np.matmul(P,b)
projected_c =np.matmul(P,c)
projected_d =np.matmul(P,d)
projected_e =np.matmul(P,e)

#Finding 2D image Coordinate of Object x' = x/z  y'=y/z
Euclidean_a = (int(projected_a[0]/projected_a[2]),int(projected_a[1]/projected_a[2]))      
Euclidean_b = (int(projected_b[0]/projected_b[2]),int(projected_b[1]/projected_b[2]))    
Euclidean_c = (int(projected_c[0]/projected_c[2]),int(projected_c[1]/projected_c[2]))
Euclidean_d = (int(projected_d[0]/projected_d[2]),int(projected_d[1]/projected_d[2]))
Euclidean_e = (int(projected_e[0]/projected_e[2]),int(projected_e[1]/projected_e[2]))

#location of points in image plane
print(Euclidean_a,Euclidean_b,Euclidean_c,Euclidean_d,Euclidean_e)
thickness =2

#connecting points in image plane by drawing line between them
img1 = cv2.line(img1, Euclidean_a, Euclidean_b, color, thickness)
img1 = cv2.line(img1, Euclidean_b, Euclidean_c, color, thickness)
img1 = cv2.line(img1, Euclidean_c, Euclidean_d, color, thickness)
img1 = cv2.line(img1, Euclidean_d, Euclidean_a, color, thickness)
img1 = cv2.line(img1, Euclidean_a, Euclidean_e, color, thickness)
img1 = cv2.line(img1, Euclidean_b, Euclidean_e, color, thickness)
img1 = cv2.line(img1, Euclidean_c, Euclidean_e, color, thickness)
img1 = cv2.line(img1, Euclidean_d, Euclidean_e, color, thickness)


cv2.imshow("Aurgumented_reality_Image",img1)
cv2.imwrite('./NewData1/Ar_output2.png',img1)
cv2.waitKey(0)


(254, 251) (302, 250) (303, 305) (254, 306) (345, 277)


-1